In [1]:
import openai
from dotenv import dotenv_values
config = dotenv_values('.env')
openai.api_key = config['OPENAI_API_KEY']

In [2]:
import pandas as pd
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
import pickle
import tiktoken

In [3]:
dataset_path = "./movie_plots.csv"
df = pd.read_csv(dataset_path)

In [5]:
movies = df[df["Origin/Ethnicity"] == "American"].sort_values("Release Year", ascending=False).head(5000)

In [6]:
movie_plots = movies["Plot"].values

In [7]:
enc = tiktoken.encoding_for_model("text-embedding-ada-002")

In [8]:
total_tokens = sum([len(enc.encode(plot)) for plot in movie_plots])

In [9]:
total_tokens
cost = total_tokens * (.0004 / 1000)
print(f"Estimated cost ${cost:.2f}")

Estimated cost $1.45


In [10]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-ada-002"):

    # replace newlines, which can negatively affect performance
    text = text.replace("\n", " ")

    return openai.Embedding.create(input=text, model=model)["data"][0]["embedding"]

In [11]:
# establsih a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding saved as a pickle file

# set path to embedding cache
embedding_cache_path = "movie_embeddings_demo.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path ,"wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embeddings_from_string(
        string,
        model="text-embedding-ada-002",
        embedding_cache=embedding_cache
):
    """Return embedding of a given a string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        print(f"GOT EMBEDDING FROM OPEN AI FOR {string[:20]}")
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]


In [12]:
plot_embeddings = [embeddings_from_string(plot, model="text-embedding-ada-002") for plot in movie_plots]

In [17]:
from nomic import atlas

ModuleNotFoundError: No module named 'nomic'